In [41]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request, urllib.parse, urllib.error
import ssl



In [42]:
# Begin initial scrape to get country names and country codes. 
# Country codes will be used to access the individual webpages fo each country.

# Scrape page

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Read the HTML from the URL and pass on to BeautifulSoup
url = 'https://www.cia.gov/library/publications/the-world-factbook/'
print("Opening the file connection...")
uh= urllib.request.urlopen(url, context=ctx)
print("HTTP status",uh.getcode())
html =uh.read().decode()
print(f"Reading done. Total {len(html)} characters read.")

soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

Opening the file connection...
HTTP status 200
Reading done. Total 75080 characters read.
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <!--<![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="css/publications.css" rel="stylesheet" type="text/css"/>
  <link href="css/publications-detail.css" rel="stylesheet" type="text/css"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   The World Factbook — Central Intelligence Agency
  </title>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="Apr 01, 2016" name="LastModified"/>
  

In [43]:
# Collect country names and country codes. Store results in lists.

country_codes=[]
country_names=[]
for tag in soup.find_all('option'):
    country_codes.append(tag.get('value')[5:7])
    country_names.append(tag.text)

temp=country_codes.pop(0) # To remove the first entry 'World'
temp=country_names.pop(0) # To remove the first entry 'World'

In [44]:
# Checking if we get expected results for country names and codes.

print('COUNTRY NAMES\n'+'-'*30)
for country in country_names[1:]:
    print(country,end=',')
print('\n\nCOUNTRY CODES\n'+'-'*30)
for country in country_codes[1:]:
    print(country,end=',')

COUNTRY NAMES
------------------------------
 Afghanistan , Akrotiri , Albania , Algeria , American Samoa , Andorra , Angola , Anguilla , Antarctica , Antigua and Barbuda , Arctic Ocean , Argentina , Armenia , Aruba , Ashmore and Cartier Islands , Atlantic Ocean , Australia , Austria , Azerbaijan , Bahamas, The , Bahrain , Baker Island , Bangladesh , Barbados , Belarus , Belgium , Belize , Benin , Bermuda , Bhutan , Bolivia , Bosnia and Herzegovina , Botswana , Bouvet Island , Brazil , British Indian Ocean Territory , British Virgin Islands , Brunei , Bulgaria , Burkina Faso , Burma , Burundi , Cabo Verde , Cambodia , Cameroon , Canada , Cayman Islands , Central African Republic , Chad , Chile , China , Christmas Island , Clipperton Island , Cocos (Keeling) Islands , Colombia , Comoros , Congo, Democratic Republic of the , Congo, Republic of the , Cook Islands , Coral Sea Islands , Costa Rica , Cote d'Ivoire , Croatia , Cuba , Curacao , Cyprus , Czechia , Denmark , Dhekelia , Djibouti 

In [45]:
# Combine country names and country codes into a dictionary

country_dict = dict(zip(country_names[1:],country_codes[1:]))
country_dict

{' Afghanistan ': 'af',
 ' Akrotiri ': 'ax',
 ' Albania ': 'al',
 ' Algeria ': 'ag',
 ' American Samoa ': 'aq',
 ' Andorra ': 'an',
 ' Angola ': 'ao',
 ' Anguilla ': 'av',
 ' Antarctica ': 'ay',
 ' Antigua and Barbuda ': 'ac',
 ' Arctic Ocean ': 'xq',
 ' Argentina ': 'ar',
 ' Armenia ': 'am',
 ' Aruba ': 'aa',
 ' Ashmore and Cartier Islands ': 'at',
 ' Atlantic Ocean ': 'zh',
 ' Australia ': 'as',
 ' Austria ': 'au',
 ' Azerbaijan ': 'aj',
 ' Bahamas, The ': 'bf',
 ' Bahrain ': 'ba',
 ' Baker Island ': 'um',
 ' Bangladesh ': 'bg',
 ' Barbados ': 'bb',
 ' Belarus ': 'bo',
 ' Belgium ': 'be',
 ' Belize ': 'bh',
 ' Benin ': 'bn',
 ' Bermuda ': 'bd',
 ' Bhutan ': 'bt',
 ' Bolivia ': 'bl',
 ' Bosnia and Herzegovina ': 'bk',
 ' Botswana ': 'bc',
 ' Bouvet Island ': 'bv',
 ' Brazil ': 'br',
 ' British Indian Ocean Territory ': 'io',
 ' British Virgin Islands ': 'vi',
 ' Brunei ': 'bx',
 ' Bulgaria ': 'bu',
 ' Burkina Faso ': 'uv',
 ' Burma ': 'bm',
 ' Burundi ': 'by',
 ' Cabo Verde ': 'cv',
 

In [46]:


# The function takes in page_data (the country page) and metrics (a list of metrics or features that we want to scrape)
# This function then creates a dictionary for the feature names and corresponding value for a given country.
# If statements are used to 5 of the metrics, because scraping these values have their own nuances.

def create_metric_dict(page_data, metrics):

    metric_list = []

    for metric in metrics:
        s = page_data.find(text=re.compile(metric))
        value = np.nan
        if s != None:
            link_container = s.parent
            div_container = link_container.parent
            # For exchange rate, we need to look two siblings ahead since one row is dedicated to explaining the type of currency
            if s == "Exchange rates:" or s == "Land boundaries:":
                value = div_container.findNextSibling().findNextSibling()
                if value != None:
                    value=value.text
                else:
                    value = np.nan
                    
            elif s == "GDP - composition, by sector of origin:":
                value = []
                for i in range (0,3):
                    div_container = div_container.findNextSibling()
                    value.append(div_container.text)
                
                if not value:
                    value = np.nan
                    
            elif s == "GDP - composition, by end use:":
                value = []
                for i in range (0,6):
                    div_container = div_container.findNextSibling()
                    value.append(div_container.text)
                    
                if not value:
                    value = np.nan
                
            else:
                value = div_container.findNextSibling().text


        metric_list.append(value)
            
    metric_dict = dict(zip(metrics,metric_list))
    
    return metric_dict



In [47]:


# country_metrics is a list of dictionaries to store the economic and societal metrics for each country.
country_metrics = []

# economic_metrics and society metrics are the metrics/features that we are interested in scraping.

economic_metrics = ["GDP - per capita \(PPP\):", \
                    "Industrial production growth rate:",\
                    "Labor force:","Unemployment rate:",\
                    "Population below poverty line:", \
                    "Distribution of family income - Gini index:", \
                    "Public debt:", "Inflation", \
                    "Current account balance:", \
                    "Exports:", "Imports:", \
                    "Reserves of foreign exchange and gold:", \
                    "Exchange rates:", \
                   "GDP - composition, by sector of origin:", \
                   "GDP - composition, by end use:"]

society_metrics = ["Population:", \
                    "Population growth rate:", \
                    "Urbanization:", \
                    "Infant mortality rate:", \
                    "Education expenditures:", \
                    "Literacy:", \
                   "Telephones - fixed lines:", \
                   "Telephones - mobile cellular:", \
                   "Internet users:", \
                    "Land boundaries:", \
                  "Geographic coordinates:"]


metric_list = economic_metrics + society_metrics

# This for loop runs through all the countries and runs the create_metric_dict function to fill out country_metrics.

for key,value in country_dict.items():
    print(key)
    url = "https://www.cia.gov/library/publications/the-world-factbook/geos/" + str(value) + ".html"
    response = requests.get(url)
    if response.status_code == 404:
        print("Unable to reach website")
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    metric_dict = create_metric_dict(soup,metric_list)
    metric_dict['Country'] = key
    country_metrics.append(metric_dict)


 Afghanistan 
 Akrotiri 
 Albania 
 Algeria 
 American Samoa 
 Andorra 
 Angola 
 Anguilla 
 Antarctica 
 Antigua and Barbuda 
 Arctic Ocean 
 Argentina 
 Armenia 
 Aruba 
 Ashmore and Cartier Islands 
 Atlantic Ocean 
 Australia 
 Austria 
 Azerbaijan 
 Bahamas, The 
 Bahrain 
 Baker Island 
 Bangladesh 
 Barbados 
 Belarus 
 Belgium 
 Belize 
 Benin 
 Bermuda 
 Bhutan 
 Bolivia 
 Bosnia and Herzegovina 
 Botswana 
 Bouvet Island 
 Brazil 
 British Indian Ocean Territory 
 British Virgin Islands 
 Brunei 
 Bulgaria 
 Burkina Faso 
 Burma 
 Burundi 
 Cabo Verde 
 Cambodia 
 Cameroon 
 Canada 
 Cayman Islands 
 Central African Republic 
 Chad 
 Chile 
 China 
 Christmas Island 
 Clipperton Island 
 Cocos (Keeling) Islands 
 Colombia 
 Comoros 
 Congo, Democratic Republic of the 
 Congo, Republic of the 
 Cook Islands 
 Coral Sea Islands 
 Costa Rica 
 Cote d'Ivoire 
 Croatia 
 Cuba 
 Curacao 
 Cyprus 
 Czechia 
 Denmark 
 Dhekelia 
 Djibouti 
 Dominica 
 Dominican Republic 
 Ecuador 
 E

In [48]:
# Test to see if copmany metrics is a list of dictionaries.

country_metrics

[{'GDP - per capita \\(PPP\\):': '$2,000 (2017 est.)',
  'Industrial production growth rate:': '-1.9% (2016 est.)',
  'Labor force:': '8.478 million (2017 est.)',
  'Unemployment rate:': '23.9% (2017 est.)',
  'Population below poverty line:': '54.5% (2017 est.)',
  'Distribution of family income - Gini index:': nan,
  'Public debt:': '7.3% of GDP (2017 est.)',
  'Inflation': '5% (2017 est.)',
  'Current account balance:': '$328 million (2017 est.)',
  'Exports:': '$784 million (2017 est.)',
  'Imports:': '$7.616 billion (2017 est.)',
  'Reserves of foreign exchange and gold:': '$7.405 billion (31 December 2016 est.)',
  'Exchange rates:': '7.87 2017 est.)',
  'GDP - composition, by sector of origin:': ['agriculture: 23%',
   'industry: 21.1%',
   'services: 55.9%'],
  'GDP - composition, by end use:': ['household consumption: 81.6%',
   'government consumption: 12%',
   'investment in fixed capital: 17.2%',
   'investment in inventories: 30%',
   'exports of goods and services: 6.7%',

In [49]:
# Convert country_metrics to DataFrame
df = pd.DataFrame(country_metrics)

# Drop some columns which we are not interested in. These were qualitative decisions.
df = df.drop(columns = ["Distribution of family income - Gini index:",\
                        "Population growth rate:", 'Exchange rates:',\
                        'Literacy:', \
                        'Reserves of foreign exchange and gold:',\
                        'Current account balance:',\
                        'Industrial production growth rate:',\
                        'Labor force:',\
                        'GDP - composition, by end use:','Inflation'])

# drop columns with too many nan's
df=df.dropna(thresh=10)
df.head()

,Country,Education expenditures:,Exports:,"GDP - composition, by sector of origin:",GDP - per capita \(PPP\):,Geographic coordinates:,Imports:,Infant mortality rate:,Internet users:,Land boundaries:,Population below poverty line:,Population:,Public debt:,Telephones - fixed lines:,Telephones - mobile cellular:,Unemployment rate:,Urbanization:
0,Afghanistan,3.2% of GDP (2015),$784 million (2017 est.),"[agriculture: 23%, industry: 21.1%, services: ...","$2,000 (2017 est.)","33 00 N, 65 00 E",$7.616 billion (2017 est.),"total: 110.6 deaths/1,000 live births","total: 3,531,770","border countries (6): China 91 km, Iran 921 km...",54.5% (2017 est.),"34,124,811 (July 2017 est.)",7.3% of GDP (2017 est.),"total subscriptions: 118,769","total subscriptions: 23,929,713",23.9% (2017 est.),urban population: 25.5% of total population (2...
2,Albania,3.5% of GDP (2015),$960.9 million (2017 est.),"[agriculture: 22.6%, industry: 23.8%, services...","$12,500 (2017 est.)","41 00 N, 20 00 E",$4.084 billion (2017 est.),"total: 11.9 deaths/1,000 live births","total: 2,016,516","border countries (4): Greece 212 km, Kosovo 11...",14.3% (2012 est.),"3,047,987 (July 2017 est.)",71.2% of GDP (2017 est.),"total subscriptions: 247,010","total subscriptions: 3,497,950",14% (2017 est.),urban population: 60.3% of total population (2...
3,Algeria,4.3% of GDP (2008),$33.15 billion (2017 est.),"[agriculture: 13.2%, industry: 36.1%, services...","$15,200 (2017 est.)","28 00 N, 3 00 E",$49.99 billion (2017 est.),"total: 19.6 deaths/1,000 live births","total: 17,291,463","border countries (7): Libya 989 km, Mali 1,359...",23% (2006 est.),"40,969,443 (July 2017 est.)",25.8% of GDP (2017 est.),"total subscriptions: 3,130,090","total subscriptions: 49,873,389",11.7% (2017 est.),urban population: 72.6% of total population (2...
4,American Samoa,NA,$428 million (2016 est.),"[agriculture: 27.4%, industry: 12.4%, services...","$11,200 (2016 est.)","14 20 S, 170 00 W",$615 million (2016 est.),"total: 11.3 deaths/1,000 live births","total: 17,000",Coastline:,NA%,"51,504 (July 2017 est.)",12.2% of GDP (2016 est.),"total subscriptions: 10,000",NaN,29.8% (2005),urban population: 87.2% of total population (2...
5,Andorra,3.3% of GDP (2016),$78.71 million (2015 est.),"[agriculture: 11.9%, industry: 33.6%, services...","$49,900 (2015 est.)","42 30 N, 1 30 E",$1.257 billion (2015 est.),"total: 3.6 deaths/1,000 live births","total: 83,887","border countries (2): France 55 km, Spain 63 km",NA%,"76,965 (July 2017 est.)",41% of GDP (2014 est.),"total subscriptions: 38,411","total subscriptions: 80,337",3.7% (2016 est.),urban population: 88.1% of total population (2...


In [50]:
# Rename columns 

df = df.rename(index=str, columns={'Education expenditures:': "education_exp", \
                              'Unemployment rate:': "unemp_rate", \
                              'Population below poverty line:': "pop_below_pov", \
                              'Inflation':'inflation',\
                              'Population:':'population', \
                              'Exports:':'exports', \
                              'Imports:':'imports', \
                              'Public debt:':'public_debt', \
                              "GDP - per capita \(PPP\):":"gdp_per_capita",\
                              'Infant mortality rate:':'infant_mortality', \
                              'Urbanization:':'urbanization', \
                              'Telephones - fixed lines:':'tel_fixed', \
                              'Telephones - mobile cellular:':'tel_cell', \
                              'Internet users:':'internet_users', \
                              'GDP - composition, by sector of origin:':'gdp_comp', \
                                  'Land boundaries:' : 'land_boundaries', \
                                  'Geographic coordinates:' : 'geo_coords'})


# Cleaning
df['education_exp'] = df['education_exp'].str.replace('%.*','')
df['unemp_rate'] = df['unemp_rate'].str.replace('%.*','')
df['public_debt'] = df['public_debt'].str.replace('%.*','').str.extract(r'([0123456789.]+)')
df['pop_below_pov'] = df['pop_below_pov'].str.replace('%.*','')
df['tel_cell'] = df['tel_cell'].str.replace(',','').str.extract(r'([\d]+)')
df['tel_fixed'] = df['tel_fixed'].str.replace(',','').str.extract(r'([\d]+)')
df['internet_users'] = df['internet_users'].str.replace(',','').str.extract(r'([\d]+)')

# Chaning NA to np.nan
df['education_exp'] = df['education_exp'].replace('NA',np.nan)
df['unemp_rate'] = df['unemp_rate'].replace('NA',np.nan)
df['pop_below_pov'] = df['pop_below_pov'].replace('NA',np.nan)

# Removing end date and commas in population
df['population'] = df['population'].str.replace('\(.*','').str.replace(',','').str.extract(r'([0123456789.]+)')
df['population'] = df['population'].replace('NA',np.nan)


# Getting rid of end dates and millions and billions
df['exports'] = df['exports'].str.replace('\(.*','').str.replace(',','')
df['imports'] = df['imports'].str.replace('\(.*','').str.replace(',','')

df["gdp_per_capita"] = df["gdp_per_capita"].str.replace('\(.*','').str.replace(',','')
df["gdp_per_capita"] = df["gdp_per_capita"].str.extract(r'(\d+)')
df["gdp_per_capita"] = df["gdp_per_capita"].replace('NA',np.nan)


# Convert strings 'millions' and 'billions' and 'trillions' to numerical millions

powers = {'billion': 10 ** 9, 'million': 10 ** 6, 'trillion' : 10 **12 , 'NaN' : 1}

def convert_millions(metric):
    extracted = metric.str.extract(r'([1234567890.]+)\s(\w+)')
    extracted.iloc[:, 1] = extracted.iloc[:, 1].map(powers).astype('float')
    out = extracted.iloc[:, 1]*extracted.iloc[:, 0].astype('float')
    return out
    
df['exports'] = convert_millions(df['exports'])
df['imports'] = convert_millions(df['imports'])

df

,Country,education_exp,exports,gdp_comp,gdp_per_capita,geo_coords,imports,infant_mortality,internet_users,land_boundaries,pop_below_pov,population,public_debt,tel_fixed,tel_cell,unemp_rate,urbanization
0,Afghanistan,3.2,7.840000e+08,"[agriculture: 23%, industry: 21.1%, services: ...",2000,"33 00 N, 65 00 E",7.616000e+09,"total: 110.6 deaths/1,000 live births",3531770,"border countries (6): China 91 km, Iran 921 km...",54.5,34124811,7.3,118769,23929713,23.9,urban population: 25.5% of total population (2...
2,Albania,3.5,9.609000e+08,"[agriculture: 22.6%, industry: 23.8%, services...",12500,"41 00 N, 20 00 E",4.084000e+09,"total: 11.9 deaths/1,000 live births",2016516,"border countries (4): Greece 212 km, Kosovo 11...",14.3,3047987,71.2,247010,3497950,14,urban population: 60.3% of total population (2...
3,Algeria,4.3,3.315000e+10,"[agriculture: 13.2%, industry: 36.1%, services...",15200,"28 00 N, 3 00 E",4.999000e+10,"total: 19.6 deaths/1,000 live births",17291463,"border countries (7): Libya 989 km, Mali 1,359...",23,40969443,25.8,3130090,49873389,11.7,urban population: 72.6% of total population (2...
4,American Samoa,NaN,4.280000e+08,"[agriculture: 27.4%, industry: 12.4%, services...",11200,"14 20 S, 170 00 W",6.150000e+08,"total: 11.3 deaths/1,000 live births",17000,Coastline:,NaN,51504,12.2,10000,NaN,29.8,urban population: 87.2% of total population (2...
5,Andorra,3.3,7.871000e+07,"[agriculture: 11.9%, industry: 33.6%, services...",49900,"42 30 N, 1 30 E",1.257000e+09,"total: 3.6 deaths/1,000 live births",83887,"border countries (2): France 55 km, Spain 63 km",NaN,76965,41,38411,80337,3.7,urban population: 88.1% of total population (2...
6,Angola,3.5,3.382000e+10,"[agriculture: 10.2%, industry: 61.4%, services...",6800,"12 30 S, 18 30 E",2.300000e+10,"total: 67.6 deaths/1,000 live births",2622403,border countries (4): Democratic Republic of t...,36.6,29310273,65.3,161070,13323952,6.6,urban population: 65.5% of total population (2...
7,Anguilla,2.8,4.200000e+06,"[agriculture: 2.4%, industry: 21.3%, services:...",12200,"18 15 N, 63 10 W",1.111000e+08,"total: 3.3 deaths/1,000 live births",13665,Coastline:,23,17087,20.1,6000,26000,8,urban population: 100% of total population (2018)
9,Antigua and Barbuda,2.5,6.130000e+07,"[agriculture: 2.3%, industry: 20.2%, services:...",26300,"17 03 N, 61 48 W",4.207000e+08,"total: 12.1 deaths/1,000 live births",60000,Coastline:,NaN,94731,86.8,22504,180000,11,urban population: 24.6% of total population (2...
11,Argentina,5.9,5.969000e+10,"[agriculture: 10.9%, industry: 28.2%, services...",20900,"34 00 S, 64 00 W",6.078000e+10,"total: 9.8 deaths/1,000 live births",30786889,"border countries (5): Bolivia 942 km, Brazil 1...",25.7,44293293,52.6,9530349,61897379,8.1,urban population: 91.9% of total population (2...
12,Armenia,2.8,2.233000e+09,"[agriculture: 17.7%, industry: 27.8%, services...",9500,"40 00 N, 45 00 E",3.361000e+09,"total: 12.7 deaths/1,000 live births",1891775,"border countries (4): Azerbaijan 996 km, Georg...",32,3045191,53.5,505190,3488524,18.9,urban population: 63.1% of total population (2...


In [51]:
# Cleaning infant mortality

def convert_infant_mortality(metric):
    extracted = metric.str.extract(r'([1234567890.]+)\D+([1234567890,]+)')
    extracted.iloc[:, 1] = extracted.iloc[:, 1].str.replace(',','')
    out = extracted.iloc[:, 0].astype('float')/extracted.iloc[:, 1].astype('float')                                  
    return out

df['infant_mortality'] = convert_infant_mortality(df['infant_mortality'])

    

In [52]:
# Set country to the index and remove whitespace

df = df.set_index('Country')
df.index = df.index.map(str.strip)


In [53]:
# Seperate GDP compositions agriculture, industry and services into 3 different columns/features.

gdp_comp_list = df['gdp_comp'].values.tolist()

for i,v in enumerate(gdp_comp_list):
    if isinstance(v, (float,)):
        gdp_comp_list[i] = [np.nan]*3

df[['agr','ind','ser']] = pd.DataFrame(gdp_comp_list, index= df.index)
df = df.drop(columns = 'gdp_comp')    

df['urbanization'] = df['urbanization'].str.extract(r'([0123456789.]+)')
df['agr'] = df['agr'].str.extract(r'([0123456789.]+)')
df['ind'] = df['ind'].str.extract(r'([0123456789.]+)')
df['ser'] = df['ser'].str.extract(r'([0123456789.]+)')
df.head()

,education_exp,exports,gdp_per_capita,geo_coords,imports,infant_mortality,internet_users,land_boundaries,pop_below_pov,population,public_debt,tel_fixed,tel_cell,unemp_rate,urbanization,agr,ind,ser
Country,,,,,,,,,,,,,,,,,,
Afghanistan,3.2,7.840000e+08,2000,"33 00 N, 65 00 E",7.616000e+09,0.1106,3531770,"border countries (6): China 91 km, Iran 921 km...",54.5,34124811,7.3,118769,23929713,23.9,25.5,23,21.1,55.9
Albania,3.5,9.609000e+08,12500,"41 00 N, 20 00 E",4.084000e+09,0.0119,2016516,"border countries (4): Greece 212 km, Kosovo 11...",14.3,3047987,71.2,247010,3497950,14,60.3,22.6,23.8,53.7
Algeria,4.3,3.315000e+10,15200,"28 00 N, 3 00 E",4.999000e+10,0.0196,17291463,"border countries (7): Libya 989 km, Mali 1,359...",23,40969443,25.8,3130090,49873389,11.7,72.6,13.2,36.1,50.7
American Samoa,NaN,4.280000e+08,11200,"14 20 S, 170 00 W",6.150000e+08,0.0113,17000,Coastline:,NaN,51504,12.2,10000,NaN,29.8,87.2,27.4,12.4,60.2
Andorra,3.3,7.871000e+07,49900,"42 30 N, 1 30 E",1.257000e+09,0.0036,83887,"border countries (2): France 55 km, Spain 63 km",NaN,76965,41,38411,80337,3.7,88.1,11.9,33.6,54.5


In [54]:
# Cleaning up bordering countries column

df['land_boundaries'] = df['land_boundaries'].str.replace(r'Coastline:','')
df['land_boundaries'] = df['land_boundaries'].str.extract(r'\:(.*)')
df['land_boundaries'] = df['land_boundaries'].str.replace(r'([0-9])','')
df['land_boundaries'] = df['land_boundaries'].str.replace(r'( km)','')
df['land_boundaries'] = df['land_boundaries'].str.replace(r'(,)','')
df['land_boundaries'] = df['land_boundaries'].str.replace(r'(\.)','')
df['land_boundaries'] = df['land_boundaries'].str.replace(r'\([^)]*\)','')
df['land_boundaries'] = df['land_boundaries'].str.split('  ')
    

df['land_boundaries']

Country
Afghanistan                 [ China, Iran, Pakistan, Tajikistan, Turkmenis...
Albania                             [ Greece, Kosovo, Macedonia, Montenegro ]
Algeria                     [ Libya, Mali, Mauritania, Morocco, Niger, Tun...
American Samoa                                                            NaN
Andorra                                                     [ France, Spain ]
Angola                      [ Democratic Republic of the Congo,  Republic ...
Anguilla                                                                  NaN
Antigua and Barbuda                                                       NaN
Argentina                       [ Bolivia, Brazil, Chile, Paraguay, Uruguay ]
Armenia                                 [ Azerbaijan, Georgia, Iran, Turkey ]
Aruba                                                                     NaN
Australia                                                                 NaN
Austria                     [ Czech Republic, Germany, H

In [55]:
# This function returns the sum of internet users in all of the bordering countries

def get_surrounding_internet(countries):
    internet_numbers = []
    
    if isinstance(countries, float):
        return np.nan      
    elif isinstance(countries, str):
        return countries
    else:
        for i in countries:
            i = i.strip()
            try:
                internet_numbers.append(float(df['internet_users'][i]))
            except KeyError:
                internet_numbers.append(0)
        return sum(internet_numbers)
    

In [56]:
# This function returns the sum of the populations of all bordering countries.

def get_surrounding_pop(countries):
    pop_numbers = []
    
    if isinstance(countries, float):
        return np.nan      
    elif isinstance(countries, str):
        return countries
    else:
        for i in countries:
            i = i.strip()
            try:
                pop_numbers.append(float(df['population'][i]))
            except KeyError:
                pop_numbers.append(0)
        return sum(pop_numbers)



In [57]:
# Create new columns for total bordering internet users and population.

df['bordering_internet'] = df['land_boundaries'].apply(lambda x:get_surrounding_internet(x))
df['bordering_pop'] = df['land_boundaries'].apply(lambda x:get_surrounding_pop(x))

In [58]:
# Create new columns to store geographic coordinates.

df['NS'] = df['geo_coords'].str.replace(',.*','')
df['EW'] = df['geo_coords'].str.replace('.*,','')

In [59]:
# This function cleans up the coordinates.

def get_coords(coords): 
    i = 1
    try :
        if coords[-1]=='S' or coords[-1]=='W':
            i = -1
        coords = coords[:-1].strip()
        coords = i*float(coords.replace(' ','.'))
        
    except (TypeError, ValueError):
        coords = np.nan
        
    return coords
    


In [60]:

# New columns for cleaned geocoordinates
df['NS_new'] = df['NS'].apply(lambda x:get_coords(x))
df['EW_new'] = df['EW'].apply(lambda x:get_coords(x))

# Scale bordering internet users by bordering population to get density
df['border_internet_scaled'] = df['bordering_internet']/df['bordering_pop']

# Drop columns we dont need
df = df.drop(columns = ['geo_coords','land_boundaries','NS', 'EW'])

df.head()

,education_exp,exports,gdp_per_capita,imports,infant_mortality,internet_users,pop_below_pov,population,public_debt,tel_fixed,...,unemp_rate,urbanization,agr,ind,ser,bordering_internet,bordering_pop,NS_new,EW_new,border_internet_scaled
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,3.2,7.840000e+08,2000,7.616000e+09,0.1106,3531770,54.5,34124811,7.3,118769,...,23.9,25.5,23,21.1,55.9,778511064.0,1.709818e+09,33.0,65.0,0.455318
Albania,3.5,9.609000e+08,12500,4.084000e+09,0.0119,2016516,14.3,3047987,71.2,247010,...,14,60.3,22.6,23.8,53.7,NaN,1.541000e+07,41.0,20.0,NaN
Algeria,4.3,3.315000e+10,15200,4.999000e+10,0.0196,17291463,23,40969443,25.8,3130090,...,11.7,72.6,13.2,36.1,50.7,NaN,9.353608e+07,28.0,3.0,NaN
American Samoa,NaN,4.280000e+08,11200,6.150000e+08,0.0113,17000,NaN,51504,12.2,10000,...,29.8,87.2,27.4,12.4,60.2,NaN,NaN,-14.2,-170.0,NaN
Andorra,3.3,7.871000e+07,49900,1.257000e+09,0.0036,83887,NaN,76965,41,38411,...,3.7,88.1,11.9,33.6,54.5,96349969.0,1.160643e+08,42.3,1.3,0.830143


In [61]:
# Create temorary data frame of just geocoordinates.

new_df = df[['EW_new','NS_new']]

In [62]:
# This loop is for find countries with the closest geocoordinates. 
# This is useful for islands which don't have bordering countries.
# The value of the internet useres of the closest country will be used if a country has no bordering countries.

country_list = new_df.index
closest_internet = {}

for country in country_list:
    distances = {}
    for i in country_list:
        ns = new_df.loc[i]['NS_new']
        ew = new_df.loc[i]['EW_new']

        d = ((ns - new_df.loc[country]['NS_new'])**2) + ((ew - new_df.loc[country]['EW_new'])**2)
        d = d**(1/2)
        distances[i] = d
        distances.pop(country, None)

    closest_country = min(distances, key=distances.get)
    closest_internet[country] = (float(df.loc[closest_country]['internet_users']))/(float(df.loc[closest_country]['population']))
    
    
df['closest_internet'] = pd.Series(closest_internet)


In [63]:
df = df.drop(columns = ['bordering_pop', 'bordering_internet', 'EW_new', 'NS_new'])
df.head()

,education_exp,exports,gdp_per_capita,imports,infant_mortality,internet_users,pop_below_pov,population,public_debt,tel_fixed,tel_cell,unemp_rate,urbanization,agr,ind,ser,border_internet_scaled,closest_internet
Country,,,,,,,,,,,,,,,,,,
Afghanistan,3.2,7.840000e+08,2000,7.616000e+09,0.1106,3531770,54.5,34124811,7.3,118769,23929713,23.9,25.5,23,21.1,55.9,0.455318,0.152928
Albania,3.5,9.609000e+08,12500,4.084000e+09,0.0119,2016516,14.3,3047987,71.2,247010,3497950,14,60.3,22.6,23.8,53.7,NaN,0.701022
Algeria,4.3,3.315000e+10,15200,4.999000e+10,0.0196,17291463,23,40969443,25.8,3130090,49873389,11.7,72.6,13.2,36.1,50.7,NaN,0.496785
American Samoa,NaN,4.280000e+08,11200,6.150000e+08,0.0113,17000,NaN,51504,12.2,10000,NaN,29.8,87.2,27.4,12.4,60.2,NaN,0.292382
Andorra,3.3,7.871000e+07,49900,1.257000e+09,0.0036,83887,NaN,76965,41,38411,80337,3.7,88.1,11.9,33.6,54.5,0.830143,0.799119


In [64]:
df.border_internet_scaled.fillna(df.closest_internet, inplace=True)
del df['closest_internet']

In [65]:
df.head()

,education_exp,exports,gdp_per_capita,imports,infant_mortality,internet_users,pop_below_pov,population,public_debt,tel_fixed,tel_cell,unemp_rate,urbanization,agr,ind,ser,border_internet_scaled
Country,,,,,,,,,,,,,,,,,
Afghanistan,3.2,7.840000e+08,2000,7.616000e+09,0.1106,3531770,54.5,34124811,7.3,118769,23929713,23.9,25.5,23,21.1,55.9,0.455318
Albania,3.5,9.609000e+08,12500,4.084000e+09,0.0119,2016516,14.3,3047987,71.2,247010,3497950,14,60.3,22.6,23.8,53.7,0.701022
Algeria,4.3,3.315000e+10,15200,4.999000e+10,0.0196,17291463,23,40969443,25.8,3130090,49873389,11.7,72.6,13.2,36.1,50.7,0.496785
American Samoa,NaN,4.280000e+08,11200,6.150000e+08,0.0113,17000,NaN,51504,12.2,10000,NaN,29.8,87.2,27.4,12.4,60.2,0.292382
Andorra,3.3,7.871000e+07,49900,1.257000e+09,0.0036,83887,NaN,76965,41,38411,80337,3.7,88.1,11.9,33.6,54.5,0.830143


In [69]:
# Convert everything into floats

df = df.astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229 entries, Afghanistan to European Union
Data columns (total 17 columns):
education_exp             179 non-null float64
exports                   220 non-null float64
gdp_per_capita            227 non-null float64
imports                   220 non-null float64
infant_mortality          224 non-null float64
internet_users            223 non-null float64
pop_below_pov             172 non-null float64
population                229 non-null float64
public_debt               210 non-null float64
tel_fixed                 222 non-null float64
tel_cell                  218 non-null float64
unemp_rate                215 non-null float64
urbanization              227 non-null float64
agr                       223 non-null float64
ind                       222 non-null float64
ser                       224 non-null float64
border_internet_scaled    221 non-null float64
dtypes: float64(17)
memory usage: 42.2+ KB


In [70]:
# Manually filling in some infoprmation. 

df.loc['Hong Kong']['internet_users'] = 6000000.0
df.loc['Taiwan']['internet_users'] = 20600000.0
df.loc['West Bank']['internet_users'] = 2600000.0
df.loc['European Union']['internet_users'] = 398000000.0
df.loc['Iran']['internet_users'] = 36000000.0
df.loc['Vietnam']['internet_users'] = 49700000.0
df.loc['Congo, Democratic Republic of the']['internet_users'] = 3000000
df.loc['Laos']['internet_users'] = 1300000

# Manually filling nans for population below poverty

df.loc['Australia']['pop_below_pov'] = 13.13
df.loc['Antigua and Barbuda']['pop_below_pov'] = 22.0
df.loc['Bahrain']['pop_below_pov'] = 80000.0/1400000.0
df.loc['Barbados']['pop_below_pov'] = 13.9
df.loc['Brunei']['pop_below_pov'] = 1000/423196.0
df.loc['British Virgin Islands']['pop_below_pov'] = 22.0
df.loc['Cayman Islands']['pop_below_pov'] = 1.9
df.loc['Cyprus']['pop_below_pov'] = 16.0
df.loc['Finland']['pop_below_pov'] = 6.0
df.loc['Guernsey']['pop_below_pov'] = 20.0
df.loc['Iceland']['pop_below_pov'] = 8.0
df.loc['Korea, North']['pop_below_pov'] = 40.0
df.loc['Kuwait']['pop_below_pov'] = 7.0
df.loc['Libya']['pop_below_pov'] = 40.0
df.loc['Liechtenstein']['pop_below_pov'] = 10.0
df.loc['Luxembourg']['pop_below_pov'] = 15.0
df.loc['Macau']['pop_below_pov'] = 10.0
df.loc['Marshall Islands']['pop_below_pov'] = 30.0
df.loc['Monaco']['pop_below_pov'] = 0.0
df.loc['Montserrat']['pop_below_pov'] = 25.0
df.loc['Nauru']['pop_below_pov'] = 24.0
df.loc['New Zealand']['pop_below_pov'] = 15.0
df.loc['Northern Mariana Islands']['pop_below_pov'] = 50.0
df.loc['Norway']['pop_below_pov'] = 7.5
df.loc['Puerto Rico']['pop_below_pov'] = 43.0
df.loc['Qatar']['pop_below_pov'] = 0.0
df.loc['Saint Kitts and Nevis']['pop_below_pov'] = 20.0
df.loc['Saint Lucia']['pop_below_pov'] = 15.0
df.loc['Saint Vincent and the Grenadines']['pop_below_pov'] = 20.0
df.loc['Samoa']['pop_below_pov'] = 18.8
df.loc['Saudi Arabia']['pop_below_pov'] = 12.7
df.loc['Singapore']['pop_below_pov'] = 20.0
df.loc['Solomon Islands']['pop_below_pov'] = 12.7
df.loc['Somalia']['pop_below_pov'] = 60.0
df.loc['Vanuatu']['pop_below_pov'] = 12.7

In [71]:
# Scale some of the columns by their populations

df['total_trade'] = df['exports']+ df['imports']
df['total_trade'] = df['total_trade']/df['population']

df['internet_users'] = df['internet_users']/df['population']

df['total_tel'] = df['tel_cell'] + df['tel_fixed']
df['total_tel'] = df['total_tel']/df['population']


df = df.drop(columns = ['exports','imports','ser','tel_fixed','tel_cell','infant_mortality'])
df = df.dropna()

In [74]:
import pickle

with open('initial_df.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [75]:
!ls

1_initial_web_scrape.ipynb find_train_size.pdf
2_feature_selection.ipynb  first_heatmap.pdf
3_modeling.ipynb           first_heatmap.svg
4_check_normality.ipynb    initial_df.pkl
First_draft.pptx           internet_heatmap.pdf
Worldfactbook.png          kids_computer.jpeg
Worldfactbook_data.png     presentation_final.pptx
find_degree.pdf            ~$presentation_final.pptx


In [76]:
df

,education_exp,gdp_per_capita,internet_users,pop_below_pov,population,public_debt,unemp_rate,urbanization,agr,ind,border_internet_scaled,total_trade,total_tel
Country,,,,,,,,,,,,,
Afghanistan,3.2,2000.0,0.103496,54.500000,34124811.0,7.3,23.9,25.5,23.0,21.1,0.455318,246.155209,0.704721
Albania,3.5,12500.0,0.661589,14.300000,3047987.0,71.2,14.0,60.3,22.6,23.8,0.701022,1655.157978,1.228667
Algeria,4.3,15200.0,0.422058,23.000000,40969443.0,25.8,11.7,72.6,13.2,36.1,0.496785,2029.317313,1.293732
Angola,3.5,6800.0,0.089470,36.600000,29310273.0,65.3,6.6,65.5,10.2,61.4,0.255319,1938.569457,0.460078
Anguilla,2.8,12200.0,0.799731,23.000000,17087.0,20.1,8.0,100.0,2.4,21.3,0.739827,6747.819980,1.872769
Antigua and Barbuda,2.5,26300.0,0.633372,22.000000,94731.0,86.8,11.0,24.6,2.3,20.2,0.540438,5088.091543,2.137674
Argentina,5.9,20900.0,0.695069,25.700000,44293293.0,52.6,8.1,91.9,10.9,28.2,0.586394,2719.824873,1.612608
Armenia,2.8,9500.0,0.621234,32.000000,3045191.0,53.5,18.9,63.1,17.7,27.8,0.320797,1836.994789,1.311482
Australia,5.2,50300.0,0.873280,13.130000,23232413.0,41.6,5.6,86.0,3.6,26.1,0.246541,18934.752925,0.000002
